In [2]:
import os, json, uuid
from datetime import datetime, timezone
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util as bsonju

# --- Load environment variables ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
database_name = os.getenv("MONGODB_DATABASE")
collection_name = "sales_invoices"

if not uri or not database_name:
    raise RuntimeError("Set MONGODB_URI and MONGODB_DATABASE in your .env")

# --- Connect to MongoDB ---
client = MongoClient(uri)
db = client[database_name]
col = db[collection_name]

# --- Helper: BSON-safe print ---
def jprint(obj):
    print(bsonju.dumps(obj, indent=2, ensure_ascii=False))

# --- Find latest EMPTY (open) document ---
latest_empty = col.find_one(
    {
        "is_empty": True,
        "$or": [
            {"items": {"$exists": False}},
            {"items": {"$size": 0}}
        ]
    },
    sort=[("created", -1)]
)

if latest_empty:
    print("✅ Reusing latest EMPTY form (no new document created).")
    jprint(latest_empty)
else:
    # --- Find latest existing document for inheritance ---
    latest_doc = col.find_one(sort=[("created", -1)])

    prev_qr = (latest_doc or {}).get("current_form_qr_code", "")
    prev_tin = (latest_doc or {}).get("tin", "")
    prev_location = (latest_doc or {}).get("location", "")

    # --- Build new empty form ---
    now_utc = datetime.now(timezone.utc)
    today_code = now_utc.strftime("%Y%m%d")          # 8-digit code, e.g., 20251015
    date_midnight_utc = datetime(
        year=now_utc.year, month=now_utc.month, day=now_utc.day, tzinfo=timezone.utc
    )

    new_doc = {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_qr,
        "current_form_qr_code": today_code,
        "tin": prev_tin,
        "location": prev_location,
        "created": now_utc,                # stored as BSON Date
        "date": date_midnight_utc,         # stored as BSON Date (00:00:00Z)
        "items": [],                       # explicitly empty
        "is_empty": True
    }

    res = col.insert_one(new_doc)
    new_doc["_id"] = res.inserted_id

    print("✅ Created new EMPTY form (no existing empty one found).")
    jprint(new_doc)


✅ Created new EMPTY form (no existing empty one found).
{
  "id": "6b4a1918-5d36-44b6-be53-1d2cbba5b64b",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "20251014",
  "tin": "123-456-789",
  "location": "JEF Gas Station – Sikatuna Branch",
  "created": {
    "$date": 1760466432321
  },
  "date": {
    "$date": 1760400000000
  },
  "items": [],
  "is_empty": true,
  "_id": {
    "$oid": "68ee9600638133c4ecb1bb6e"
  }
}
